In [ ]:

import numpy as np
import pandas as pd
import sklearn
import os



#These libraries help with dealing with image datasets
import cv2
import matplotlib.pyplot as plt
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torchvision import datasets, models, transforms

!pip install random2
import random2
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
!pip install kneed
from kneed import KneeLocator
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

import faiss
import glob
import seaborn as sns
import matplotlib.cm as cm
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
with open('C:/Users/laura/Google Drive/WikiArt2/features.npy', 'rb') as f:
    X = np.load(f)
with open('C:/Users/laura/Google Drive/WikiArt2/labels.npy', 'rb') as f:
    y = np.load(f)

In [ ]:
import gc

gc.collect()

In [ ]:
# pre-process

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.7)

print(X_train.shape)
print(X_train.reshape(len(X_train),-1).shape)

# flatten 128*128*3
reshaped_data = X_train.reshape(len(X_train),-1)
# normalise
gc.collect()
# convert from integers to floats
reshaped_data = reshaped_data.astype('float16')
# normalize to the range 0-1
reshaped_data /= 255.0
# confirm the normalization
print('Min: %.3f, Max: %.3f' % (reshaped_data.min(), reshaped_data.max()))

In [ ]:
reshaped_y = y_train.astype('int16')

In [ ]:
!pip install datatable
import datatable as dt
reshape_dt = dt.Frame(reshaped_data)

In [ ]:
# mapping labels from cluster to original labels
def get_reference_dict(clusters,data_label):
    reference_label = {}
    # For loop to run through each label of cluster label
    for i in range(len(np.unique(clusters))):
        index = np.where(clusters == i,1,0)
        num = np.bincount(data_label[index==1]).argmax()
        reference_label[i] = num
    return reference_label
# Mapping predictions to original labels
def get_labels(clusters,reference_labels):
    temp_labels = np.random.rand(len(clusters))
    for i in range(len(clusters)):
        temp_labels[i] = reference_labels[clusters[i]]
    return temp_labels

In [ ]:
from sklearn.cluster import MiniBatchKMeans

In [ ]:
k = 14
kmeans = MiniBatchKMeans(
    n_clusters=k, 
    random_state=12, 
    batch_size=1000)
clusters = kmeans.fit_predict(reshaped_data)
clusters = clusters.astype("int")
sse=kmeans.inertia_
reference_labels = get_reference_dict(clusters,y_train)
predicted_labels = get_labels(clusters,reference_labels)
print(f"Accuracy for k = {k}: ", accuracy_score(predicted_labels,y_train))

In [ ]:
plt.hist(clusters)
plt.title('Number of classifications per cluster');

In [ ]:
from sklearn.manifold import TSNE
idx = np.random.randint(6000, size=1500)
reshaped_data_sub = reshaped_data[idx,:]
clusters_sub = clusters[idx]
cmap = plt.get_cmap('cool',14) 


def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)
    
    u_labels = np.unique(y1)
    
    plt.figure(figsize=(12, 8))
    for i in u_labels:
        plt.scatter(X_t[np.where(y1 == i), 0], X_t[np.where(y1 == i), 1], linewidth='1', alpha=0.8, label= i, cmap =cmap)
    plt.legend(loc='best');
    plt.savefig(name);
    plt.show();
    
tsne_plot(reshaped_data_sub, clusters_sub, "original.png")

In [ ]:
reference_labels = get_reference_dict(clusters,y_train)
predicted_labels = get_labels(clusters,reference_labels)
print(accuracy_score(predicted_labels,y_train))

In [ ]:
gc.collect()


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.9, random_state=12, shuffle=True, stratify=y)

In [ ]:
gc.collect()
sse = []
list_k = [14, 32,64,96,128,160,256]
for k in list_k:
    kmeans = MiniBatchKMeans(
        n_clusters=k, 
        random_state=12, 
        batch_size=1000)
    clusters = kmeans.fit_predict(reshaped_data)
    clusters = clusters.astype("int")
    sse.append(kmeans.inertia_)
    reference_labels = get_reference_dict(clusters,y_train)
    predicted_labels = get_labels(clusters,reference_labels)
    print(f"Accuracy for k = {k}: ", accuracy_score(predicted_labels,y_train))
#end for

In [ ]:
# Plot sse against k
plt.figure(figsize=(6, 6))
plt.plot(list_k, sse, '-o')
plt.xlabel(r'Number of clusters')
plt.ylabel('Sum of squared distance');

In [ ]:
gc.collect()

In [ ]:
k = 64
kmeans = MiniBatchKMeans(
    n_clusters=k, 
    random_state=12, 
    batch_size=1000)
clusters = kmeans.fit_predict(reshaped_data)
clusters = clusters.astype("int")
sse=kmeans.inertia_
reference_labels = get_reference_dict(clusters,y_train)
predicted_labels = get_labels(clusters,reference_labels)
print(f"Accuracy for k = {k}: ", accuracy_score(predicted_labels,y_train))

In [ ]:
plt.hist(clusters)
plt.title('Number of classifications per cluster');

In [ ]:
plt.hist(predicted_labels)
plt.title('Number of classifications per style');

In [ ]:
from sklearn.manifold import TSNE
idx = np.random.randint(14000, size=1500, replace = False)
reshaped_data_sub = reshaped_data[idx,:]
clusters_sub = clusters[idx]

def tsne_plot(x1, y1, name="graph.png"):
    tsne = TSNE(n_components=2, random_state=0)
    X_t = tsne.fit_transform(x1)
    
    u_labels = np.unique(y1)
    
    plt.figure(figsize=(12, 8))
    for i in u_labels:
        plt.scatter(X_t[np.where(y1 == i), 0], X_t[np.where(y1 == i), 1], linewidth='1', alpha=0.8, label= i)
    plt.legend(loc='best');
    plt.savefig(name);
    plt.show();
    
tsne_plot(reshaped_data_sub, clusters_sub, "original.png")

In [ ]:
gc.collect()